### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


* Display the total number of players


In [3]:
#Unique count to filter duplicate players and display in Dataframe
TotalPlayer = purchase_data["SN"].nunique()
TotalPlayers = pd.DataFrame({"Total Players": TotalPlayer}, index=[0])
TotalPlayers

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#Create DataFrame with 4 columns, using a dictionary with keys and values based on agg functions. 
SummaryDF = pd.DataFrame({"Number of Unique Items": [purchase_data["Item ID"].nunique()], 
                                "Average Price": [purchase_data["Price"].mean()],
                                "Number of Purchases": [purchase_data["Purchase ID"].count()],
                                "Total Revenue": [purchase_data["Price"].sum()]})

SummaryDF = SummaryDF.style.format({"Average Price": "${:,.2f}", "Total Revenue": "${:,.2f}"})
SummaryDF

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

In [5]:
#Create Dataframe with Gender as index, find total amount of players for each gender, removing duplicates. 
GenderDemo = purchase_data.drop_duplicates(subset="SN", keep = "first")
GenderDemo = GenderDemo.groupby("Gender").count()
GenderDemo = GenderDemo.iloc[:, 0:2]
GenderDemo.rename(columns={"Purchase ID": "Total Count", "SN":"% of Players"}, inplace=True)
GenderDemo["% of Players"] = GenderDemo["Total Count"]/TotalPlayer
#Formatting
GenderDemo = GenderDemo.rename_axis("Index").sort_values(by = ["Total Count", "Index"], ascending=[False, True])
GenderDemo = GenderDemo.style.format({"% of Players": "{:.2%}"})
GenderDemo

,Total Count,% of Players
Index,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed





## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
#Get Purchase Count, Average Purchase Price, Total Purchase Price
GenderPurchase = purchase_data.groupby("Gender").agg({"SN": ["count"], "Price": ["mean", "sum"]})
#Rename columns
GenderPurchase.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
#Drop dups for Total Purch PER PERSON
NoDups = purchase_data.drop_duplicates(subset="SN", keep = "first")
NoDups = NoDups["Gender"].value_counts()
#Get Total by gender
TotalPur = purchase_data.groupby("Gender")["Price"].sum()
#equation
ATP = TotalPur/NoDups
#add to DF
GenderPurchase["Avg Total Purchase Per Person"] = ATP
#Format
GenderPurchase = GenderPurchase.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:,.2f}", "Avg Total Purchase Per Person":"${:,.2f}"})
GenderPurchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
#Drop dups so peeps show up once
AgeDemo = purchase_data.drop_duplicates(subset="SN", keep = "first")
#my bins, my boxes, my categories
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 1000]
#My labels
labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#Add to DF
AgeDemo["Age Demo"] = pd.cut(AgeDemo["Age"], bins, labels=labels, include_lowest=True)
#Get new index
AgeDemo = AgeDemo.groupby("Age Demo")
#Cull some columns, just need 1 count
AgeDemo = AgeDemo.count().iloc[:, 0:2]
#Rename
#AgeDemo.rename(columns={"Purchase ID": "Total Count", "SN": "% of Players"}, inplace=True )
AgeDemo.columns=[ "Total Count",  "% of Players"]
#Equation to get %
AgeDemo["% of Players"] = (AgeDemo["Total Count"]/TotalPlayer)*100
#Formatting
AgeDemo = AgeDemo.style.format({"% of Players":"{:,.2f}%"})
AgeDemo

,Total Count,% of Players
Age Demo,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [13]:
#my bins, my boxes, my categories
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 1000]
#My labels
labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#Add to DF
PurAge = purchase_data
PurAge["Age Demo"] = pd.cut(PurAge["Age"], bins, labels=labels, include_lowest=True)
####################################
####Need this variable for later##
TotalAgePurchase = PurAge.groupby("Age Demo")["Price"].sum()
######################################

#Manipulate data and change DF index
PurAge = PurAge.groupby("Age Demo").agg({"SN":["count"], "Price":["mean", "sum"]})
#Rename DF columns
PurAge.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]

#Avg Total Purchase Per Person
#Drop dups so peeps show up once
PerPerson = purchase_data.drop_duplicates(subset="SN", keep = "first")
PerPerson["Age Demo"] = pd.cut(PerPerson["Age"], bins, labels=labels, include_lowest=True)
OneAge = PerPerson["Age Demo"].value_counts()
ATP2 = TotalAgePurchase/OneAge
PurAge["Avg Total Purchase Per Person"] = ATP2
#Formatting
PurAge = PurAge.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:,.2f}", "Avg Total Purchase Per Person":"${:,.2f}"})
PurAge

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Demo,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
TopSpenders = purchase_data.groupby("SN").agg({"SN":["count"], "Price": ["mean", "sum"]})
TopSpenders.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
TopSpenders.sort_values(by=["Total Purchase Value"], ascending=False, inplace=True)
TopSpenders = TopSpenders.iloc[0:5,:]
TopSpenders = TopSpenders.style.format({"Average Purchase Price": "${:,.2f}", "Total Purchase Value": "${:,.2f}"})
TopSpenders


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
PopItem = purchase_data.groupby(["Item ID","Item Name"]).agg({"Price": ["count", "first", "sum"]})
PopItem.columns = ["Purchase Count",  "Item Price", "Total Purchase Value"]
PopItem.sort_values(by=["Purchase Count"], inplace=True, ascending=False)
PopItem = PopItem.iloc[0:5,:]
PopItem = PopItem.style.format({"Item Price": "${:,.2f}","Total Purchase Value":"${:,.2f}"})
PopItem

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [8]:
ProfitItem = purchase_data.groupby(["Item ID","Item Name"]).agg({"Price": ["count", "first", "sum"]})
ProfitItem.columns = ["Purchase Count",  "Item Price", "Total Purchase Value"]
ProfitItem.sort_values(by=["Total Purchase Value"], inplace=True, ascending=False)
ProfitItem = ProfitItem.iloc[0:5,:]
ProfitItem = ProfitItem.style.format({"Item Price": "${:,.2f}","Total Purchase Value":"${:,.2f}"})
ProfitItem

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
